# Notebook used to create the dictionary of the names
We used 3 sources of data do create this dictionary, the labels collected by:
* Bartoletti et al.(2018)
* Meikeljon et al.(2013)
* Web scrapping did by the students
* Blockchain.com

The data are stored in the folder /mnt/DD2/bitcoinJson2016  
In the following lines we are:
- first loading the csv files containing the data as a Pandas DataFrame
- Transforming each DataFrame to have 2 columns : name and addresses
- Each row contains one name and one address
- Then we append all DataFrame, and convert it to a dictionary
- We gather all the data and build one dictionary of the labels


In [ ]:
# modules
import os
import pandas as pd
from functions_check_in_data import *
from functions_extract_transform_data import *
from functions_write_save_load_data import *

###### LIST ADDRESSES 1
Source : Bartoletti et al.(2018)  
Link : https://github.com/bitcoinponzi/BitcoinPonziTool/tree/master/CSV/Address

In [ ]:
# charge ADDRESSES #1
data_1 = pd.read_csv("addresses1.csv", skipinitialspace=True)
data_1[0:10]

###### LIST ADDRESSES 2
Source : Bartoletti et al.(2018)  
Link : https://github.com/bitcoinponzi/BitcoinPonziTool/tree/master/CSV/Address

In [ ]:
data_2 = pd.read_csv("addresses2.csv", skipinitialspace=True, usecols=[0, 1])
data_2.iloc[1:10]

In [ ]:
# Some names in data_2 have multiple addresses in one line
# Let put each of those addresses in one line
from itertools import chain

df = data_2


# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split("-")))


# calculate lengths of splits
lens = df["addresses"].str.split("-").map(len)

# create new dataframe, repeating or chaining as appropriate
data_2 = pd.DataFrame({
    "name": np.repeat(df["name"], lens),
    "addresses": chainer(df["addresses"])
})
data_2[:10]

###### LIST ADDRESSES 3
Source : Bartoletti et al.(2018)  
Link : https://github.com/bitcoinponzi/BitcoinPonziTool/tree/master/CSV/Address

In [ ]:
#  Charge data
data_3 = pd.read_csv("addresses3.csv", usecols=[0, 1], skipinitialspace=True)
data_3.iloc[25:35]

###### LIST ADDRESSES 4
This list of addresses was collected from : https://www.blockchain.com/btc/popular-addresses. 

In [ ]:
data_4 = pd.read_csv("addresses4.csv", skipinitialspace=True, usecols=[0, 1])
data_4.iloc[30:35]

###### MEIKLEJOHN ADDRESSES
This list addresses was collected by Meiklejohn and al.(2013)

In [ ]:
data_5 = pd.read_csv("meiklejohn.csv", header=None, usecols=[2, 3])
data_5.columns = ["addresses", "name"]
data_5 = data_5.drop_duplicates(keep=False)

data_5[:10]

###### Create a dictionary with 5 DataFrames

In [ ]:
# Append all the dataframes into one
adds_names = pd.concat([data_1, data_2, data_3, data_4, data_5])
adds_names[:10]

In [ ]:
# Total number of addresses obtained with the
# Previous 5 DataFrames
len(adds_names)

In [ ]:
# Load the dictionary of addresses
addresses_ids = load_dumped_data("addresses_ids.pickle", "pickle")

# and convert the addresses column of adds_name to integers
adds_names["addresse_id"] = force_rewrite(adds_names["addresses"], addresses_ids)
adds_names[20:25]

In [ ]:
# Transform the DataFrame adds_names to a dictionary
# where : the keys are addresses as integers, the
# Values are the labels
dictionary_names_int = pd.Series(adds_names.name.values,
                                 index=adds_names.addresse_id).to_dict()
del dictionary_names_int["NA"]
dictionary_names_int

In [ ]:
# Total number of available labels in the data (until 2016)
len(dictionary_names_int)

In [ ]:
# A second dictionary where btc addresses are the keys
# and the names are values.
dictionary_names_str = pd.Series(adds_names.name.values,
                                 index=adds_names.addresses).to_dict()
# dictionary_names_str

In [ ]:
# Total number of available labels in the data (until 2019)
len(dictionary_names_str)

######  LIST ADDRESSES BTTALK
Web scrapping did by the Students. 

In [ ]:
#  Add the addresses from bttalk
import json

with open("bttalk.json", "r") as f:
    bttalk_adds = json.load(f)
bttalk_adds["178aFfkk6eVr4nqnbfRX2f5zMh3r9zb2TH"]["Name"], len(bttalk_adds)

In [ ]:
# Add all the BTC addresses to dictionary_names_str
for i, j in bttalk_adds.items():
    dictionary_names_str[i] = j["Name"]

In [ ]:
# New number of total BTC addresses and their labels (until 2019)
len(dictionary_names_str)

In [ ]:
# save the dictionary of names: dictionary_names_str on the hard drive
dump_variable(dictionary_names_str,
              "dictionary_of_btc_names_str.pickle", "pickle")

In [ ]:
# Create a dictionary of the bttalk addresses
keys_adds = []  # The keys are the btc addresses
values_names = []  # The values are the names
for i, j in bttalk_adds.items():
    keys_adds.append(i)
    values_names.append(j["Name"])

# Convert the btc addresses to integers
keys_adds = force_rewrite(keys_adds, addresses_ids)

bttalk_adds_dic = dict(zip(keys_adds, values_names))
del bttalk_adds_dic["NA"]

In [ ]:
# Number of addresses from bttalk
len(bttalk_adds_dic)

In [ ]:
# Add the addresses from bttalk_adds_dic to the dictionary_names_int
conflict_adds = []  # Addresses present in both dictionaries
for i, j in bttalk_adds_dic.items():
    # If the address from bttalk is already in dictionary_names_int
    if i in dictionary_names_int.keys():
        print(i, dictionary_names_int[i])
        conflict_adds.append(i)
    dictionary_names_int[i] = j

In [ ]:
# print the names of addresses that are in both dictionaries
# check if they have the same names
for i in conflict_adds:
    print(bttalk_adds_dic[i])
    print(dictionary_names_int[i])

In [ ]:
# conflict_adds

In [ ]:
# size of the new dictionary with all addresses in integers
# and their labels
len(dictionary_names_int)

In [ ]:
# save the dictionary of names: dictionary_names_int on the hard drive
dump_variable(dictionary_names_int,
              "dictionary_of_btc_names_int.pickle", "pickle")